In [312]:
# imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

pd.options.plotting.backend = "plotly"

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet import Prophet
import itertools
from sktime.performance_metrics.forecasting import (
    mean_absolute_scaled_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
)
import time
import holidays
import datetime


# define functions

In [313]:
horizon = 7

def preprocessing_endo(ts, steps_ahead=horizon, split=True):
    fb_df = ts[["sales"]]
    fb_df["sales"] = ts["sales"] / 1e6
    fb_df = fb_df.reset_index().rename({"date": "ds", "sales": "y"}, axis=1)

    if split:
        fb_train = fb_df.iloc[:-steps_ahead]
        fb_test = fb_df.iloc[-steps_ahead:]
        return {"y_train": fb_train, "y_test": fb_test}
    else:
        return fb_df


def preprocessing_exog(df_exog):
    df_promo = pd.DataFrame({
        "holiday": "promo_day",
        "ds": df_exog[df_exog["promo_day"] == 1]["promo_day"].index})
    df_off = pd.DataFrame({
        "holiday": "off_day",
        "ds": df_exog[df_exog["off_day"] == 1]["off_day"].index})
    return pd.concat([df_promo, df_off])


# grid seach Prophet
def grid_search_prophet(prophet_data_endo, prophet_data_exog, param_grid):
    # create df to store result
    mape_PROPHET = pd.DataFrame(columns=["params", "mape"])
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

    # grid search
    y_train = prophet_data_endo['y_train']
    y_test = prophet_data_endo['y_test']
    for params in all_params:
        m = Prophet(holidays=prophet_data_exog,
                    **params).fit(y_train)  # Fit model with given params
        y_hat = m.predict(y_test)['yhat']
        mape_PROPHET = mape_PROPHET.append(
            {"params": params, 
             "mape": round(mean_absolute_percentage_error(y_test['y'], y_hat), 3)}, ignore_index=True)
    mape_PROPHET.sort_values('mape', inplace=True)
    return mape_PROPHET


# CV 
def cross_validation_result(data, exog, params, model_name, rolls=4, horizon=horizon):
    """ """
    mae_CVs = []
    rmse_CVs = []
    mape_CVs = []
    mase_CVs = []
    for i in range(rolls):
        print(f"fold {i}---------------")
        
        # split data
        y_train = data.iloc[: -(rolls - i) * horizon]
        y_test = data.iloc[
            np.r_[-(rolls - i) * horizon : -(rolls - i - 1) * horizon]]

        # fit model
        model = Prophet(**params, holidays=exog)
        model.fit(y_train)
        
        # make forecast
        y_hat = model.predict(y_test)

        # score
        mae_CVs.append(round(mean_absolute_error(y_test['y'], y_hat['yhat']), 3))
        rmse_CVs.append(round(mean_squared_error(y_test['y'], y_hat['yhat'], square_root=True), 3))
        mape_CVs.append(round(mean_absolute_percentage_error(y_test['y'], y_hat['yhat']), 3))
        mase_CVs.append(round(mean_absolute_scaled_error(y_test['y'], y_hat['yhat'], y_train=y_train['y']), 3))
    
    return {'store':model_name,
           'mae_PROPHET':np.mean(mae_CVs),
           'rmse_PROPHET':np.mean(rmse_CVs),
           'mape_PROPHET':np.mean(mape_CVs),
           'mase_PROPHET':np.mean(mase_CVs),
           'fc_PROPHET':y_hat,
           }



# read data

In [314]:
df_store = pd.read_pickle("data/df_daily.pkl")
df_company = df_store.groupby("date").sum()[["sales"]]
df_exog = pd.read_pickle("data/df_exog.pkl")

df_company_pro = preprocessing_endo(df_company, split=True)
df_exog_pro = preprocessing_exog(df_exog)


# tune on company data

In [315]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.005, 0.01, 0.05, 0.1],
    "seasonality_prior_scale": [1.0, 0.01, 0.1, 1.0],
    "daily_seasonality": [False],
    "weekly_seasonality": [True],
    "yearly_seasonality": [True],
}



## grid search
- fit once per param set on train set
- predict & score on test set (no CV)

In [316]:
grid_search_result = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        2771.6   1.61909e-05       658.564       0.362      0.5238      140   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     127       2771.61   5.45983e-07       727.442   8.534e-10       0.001      222  LS failed, Hessian reset 
     137       2771.61   2.74154e-08       607.669      0.3379           1      238   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2751.01    4.5142e-06       784.208      0.9424      0.9424      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     133       2751.17   7.84244e-05       585.529   1.493e-07       0.001      222  LS failed, Hessian reset 
     184       2751.29   1.22653e-08       520.782      0.2993     0.02993      296   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2810.42    0.00123975       66.2777     0.09184           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2811.15    0.00013307       43.4635           1           1      262   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     284       2811.41   2.31728e-05       33.2451   6.282e-07       0.001      413  LS failed, Hessian reset 
     299       2811.41   1.16724e-06       31.9109      0.3393           1      436   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     306       2811.41   1.45621e-07       36.7189       0.085           1      448   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2810.66   0.000131651       35.0282           1           1      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     118       2810.81    9.8021e-05       48.5456   1.759e-06       0.001      195  LS failed, Hessian reset 
     199       2811.37   0.000687513       43.3282           1           1      296   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2811.71   5.00675e-05       40.3363      0.5708      0.5708      421   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2811.86    9.5884e-05       35.1029     0.03466     0.03466      544   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     484       2811.97   0.000107295    

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(


In [317]:
grid_search_result


,params,mape
13,"{'changepoint_prior_scale': 0.1, 'seasonality_...",0.115
2,"{'changepoint_prior_scale': 0.005, 'seasonalit...",0.12
12,"{'changepoint_prior_scale': 0.1, 'seasonality_...",0.12
15,"{'changepoint_prior_scale': 0.1, 'seasonality_...",0.12
14,"{'changepoint_prior_scale': 0.1, 'seasonality_...",0.122
0,"{'changepoint_prior_scale': 0.005, 'seasonalit...",0.138
3,"{'changepoint_prior_scale': 0.005, 'seasonalit...",0.138
9,"{'changepoint_prior_scale': 0.05, 'seasonality...",0.143
10,"{'changepoint_prior_scale': 0.05, 'seasonality...",0.148
8,"{'changepoint_prior_scale': 0.05, 'seasonality...",0.149


In [318]:
tuned_prophet_params = grid_search_result.iloc[0, 0]
tuned_mape = grid_search_result.iloc[0, 1]
tuned_mape, tuned_prophet_params


(0.115,
 {'changepoint_prior_scale': 0.1,
  'seasonality_prior_scale': 0.01,
  'daily_seasonality': False,
  'weekly_seasonality': True,
  'yearly_seasonality': True})

# fit on store data

In [319]:
all_stores_result_CV = []
for store in df_store["store_id"].unique():#[:2]:
    print(f"processing stores {store}...\n")
    model_name = "store_" + str(store)

    # data
    df_1_store = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_1_store_pro = preprocessing_endo(df_1_store, split=False)

    # define model
    # model_store = Prophet(**tuned_prophet_params, holidays=df_exog_pro)
    
    # score model with CV on store data
    cv_score = cross_validation_result(df_1_store_pro, df_exog_pro, tuned_prophet_params, model_name=model_name)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


processing stores 307222...

fold 0---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -7.38369
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2759.91   7.61095e-05       49.7687       0.404       0.404      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     138       2760.22   0.000168839       66.5303   4.704e-06       0.001      228  LS failed, Hessian reset 
     199        2760.3   0.000129406       31.2291      0.7973      0.7973      300   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     275       2760.34     5.129e-07       26.1187      0.2196      0.8885      408   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


fold 1---------------


...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


fold 3---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -17.4505
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1376.65     0.0013799       43.8279      0.7903      0.7903      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     143       1378.26   0.000450499        46.174   4.033e-06       0.001      213  LS failed, Hessian reset 
     199       1378.75   5.85531e-06       37.6958           1           1      285   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       1379.27   0.000145741       42.3985           1           1      410   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     388       1379.59   0.000347339       38.3813   8.456e-06       0.001      602  LS failed, Hessian reset 
     399       1379.63   0.000121736       37.0774      0.4652      0.4652      616   
    Iter      log pro

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


# result

In [320]:
all_stores_result_CV.to_pickle("results/f8/PROPHET_7.pkl")


In [321]:
all_stores_result_CV.head()


,store,mae_PROPHET,rmse_PROPHET,mape_PROPHET,mase_PROPHET,fc_PROPHET
0,store_307222,7.07675,8.79800,0.23400,0.71600,ds trend yhat_lower yhat_uppe...
1,store_307244,6.48750,8.06575,0.30900,0.83200,ds trend yhat_lower yhat_uppe...
2,store_307248,5.43775,6.67150,0.23900,0.79100,ds trend yhat_lower yhat_uppe...
3,store_320264,4.64275,7.18550,0.26875,0.68500,ds trend yhat_lower yhat_upper...
4,store_328165,27.24225,36.93400,0.35700,1.24275,ds trend yhat_lower yhat_uppe...


In [322]:
all_stores_result_CV.describe()


,mae_PROPHET,rmse_PROPHET,mape_PROPHET,mase_PROPHET
count,38.000000,38.000000,38.000000,38.000000
mean,7.010829,8.870763,0.338375,0.951572
std,5.261995,7.191575,0.098701,0.203832
min,2.873500,3.770750,0.234000,0.588000
25%,3.747187,4.523313,0.263375,0.808187
50%,5.513500,6.928500,0.309500,0.917000
75%,7.431750,9.393188,0.380937,1.064188
max,27.242250,36.934000,0.647000,1.539000
